In [48]:
import os
import shutil
import pickle
import string
import pandas as pd
from keras_preprocessing.text import Tokenizer

Так как количество данных для перевода было огромно а все апи по машинному переводу накладывают кучу ограничений
Яндекс - не более 20 запросов в минуту и не более миллиона символов за час. Плюс у тебя должен быть биллинг аккаунт с привязанной картой и эта функция платная
Гугл - не более 2 миллионов символов за 24 часа и не более  50 запросов в час. Платная функция
promt - не более 900 символов за раз, апи платное

Были также рассмотерны возможности прокси серверов
   - бесплатные прокси очень редко поддерживают https (а все апи по переводу работают через него)
   - scrapper API: поддерживает всего 5000 запросов в месяц

Было решено перевести csv файл с английским описанием в docx средствами Microsoft Word.
Далее этот файл был загружен через форму перевода в translate.google и переведн примерно за 2 минуту
После чего файл был обратно пересохранен в csv
Именно с этим файлом мы и будем дальше работать

In [17]:
dataset_path = "D:/Temp/Dataset/kaggle/flickr_30k" # основной путь к датасету
path_ru_captions = dataset_path + "/ru_labels.csv"

df = pd.read_csv(path_ru_captions, delimiter='|')
df.head(5)

,image_name,comment_number,comment
0,1000092795.jpg,0,два молодых парня с лохматыми волосами смотрят...
1,1000092795.jpg,1,два молодых белых мужчины стоят на улице возле...
2,1000092795.jpg,2,двое мужчин в зеленых рубашках стоят во дворе
3,1000092795.jpg,3,мужчина в синей рубашке стоит в саду
4,1000092795.jpg,4,"два друга наслаждаются временем, проведенным в..."


### Для работы с текстовым описанием изображений необходимо очистить данные от шума

In [18]:
# Воспользуемся функцией очистки текста из файа 1_prepare_data
# Для того, чтобы уменьшить размер словесного запаса с которым придется работать
# - все слова в нижний регистр
# - удаляем все знаки препинания
# - удаляем все лова длиной один символ или меньше
# - удаляем все слова с цифрами в них
def clean_text(data):
    idx_to_delete = [] # массив в котором будем хранить индексы для битых строк, которые
                       # надо удалить
    table = str.maketrans('', '', string.punctuation)
    for idx in data.index:
        if pd.notna(data.iat[idx, 2]): # если существует описание изображения
            # разбиваем описание на слова
            desc = data.iat[idx, 2].split()
            # - все лова в нижний регистр
            desc = [word.lower() for word in desc]
            # - удаляем все знаки препинания
            desc = [word.translate(table) for word in desc]
            # - удаляем все слова длиной один символ или меньше
            desc = [word for word in desc if (len(word) > 1)]
            # - удаляем все слова с цифрами в них
            desc = [word for word in desc if (word.isalpha())]
            # преобразуем обратно в строку
            data.iat[idx, 2] = ' '.join(desc)
        else: # если не существует описания изображения, добавляем индекс в массив для удаления
            idx_to_delete.append(idx)
    return idx_to_delete

### Запускаем очистку текста

In [19]:
idxs_to_delete = clean_text(df) # сохраним индексы с битыми описаниями в переменную
df.head(10) # выведем первые 10 строк из датасета

,image_name,comment_number,comment
0,1000092795.jpg,0,два молодых парня лохматыми волосами смотрят н...
1,1000092795.jpg,1,два молодых белых мужчины стоят на улице возле...
2,1000092795.jpg,2,двое мужчин зеленых рубашках стоят во дворе
3,1000092795.jpg,3,мужчина синей рубашке стоит саду
4,1000092795.jpg,4,два друга наслаждаются временем проведенным вм...
5,10002456.jpg,0,несколько мужчин касках управляют системой гиг...
6,10002456.jpg,1,рабочие смотрят сверху вниз на оборудование
7,10002456.jpg,2,двое мужчин работают на машине касках
8,10002456.jpg,3,четыре человека на вершине высокой конструкции
9,10002456.jpg,4,трое мужчин на большой платформе


In [20]:
# проверим какова длина массива с битыми строками
print(len(idxs_to_delete))

# возьмем все строки для удаления
df_to_delete = df.loc[idxs_to_delete, :]
# возьмем уникальное имя изображения, т.к. будем удалять все описания для этих изображений
images_to_delete = df_to_delete["image_name"].unique()
# распечатаем имена изображений для удаления
print(images_to_delete)

0
[]


In [21]:
# удалим все строки с этим изображением
df = df[~df.image_name.isin(images_to_delete)]
# Сохраним датафрейм в файл
path_prepared_captions = dataset_path + "/clean_results_ru.csv"
df.to_csv(path_prepared_captions, sep='|', encoding='utf-8', index=False)

### Разбить набор на три части
Для обучения необходимо разбить набор на три части:
  - обучающая выборка
  - валидационная выборка
  - тестовая выборка

In [22]:
n = len(df)

train_df = df[0:int(n*0.7)]
val_df = df[int(n*0.7):int(n*0.9)]
test_df = df[int(n*0.9):]

path_train = dataset_path + "/ru-train.csv"
path_val = dataset_path + "/ru-val.csv"
path_test = dataset_path + "/ru-test.csv"

train_df.to_csv(path_train, sep='|', encoding='utf-8', index=False)
val_df.to_csv(path_val, sep='|', encoding='utf-8', index=False)
test_df.to_csv(path_test, sep='|', encoding='utf-8', index=False)

Модель, которую мы разработаем, будет генерировать подпись к фотографии, и подпись будет генерироваться по одному слову за раз.
Последовательность ранее сгенерированных слов будет предоставлена в качестве входных данных. Поэтому нам понадобится “первое слово”,
 чтобы начать процесс генерации, и “последнее слово”, чтобы сигнализировать об окончании подписи. Для этой цели мы будем использовать
  строки “start” и “end”.
  Эти маркеры добавляются к загруженным описаниям по мере их загрузки. Важно сделать это сейчас, прежде чем мы закодируем текст,
  чтобы токены также были закодированы правильно.

In [23]:
def add_start_end_tags(data):
    for idx in data.index:
        curr_str = data.iat[idx, 2]
        data.iat[idx, 2] = 'start ' + curr_str + ' end'

In [24]:
train_df = pd.read_csv(path_train, sep='|')
val_df = pd.read_csv(path_val, sep='|')
test_df = pd.read_csv(path_test, sep='|')

add_start_end_tags(train_df)
add_start_end_tags(val_df)
add_start_end_tags(test_df)

train_df.to_csv(path_train, sep='|', encoding='utf-8', index=False)
val_df.to_csv(path_val, sep='|', encoding='utf-8', index=False)
test_df.to_csv(path_test, sep='|', encoding='utf-8', index=False)

In [25]:
# проверим что у нас получилось
test_df.iat[0, 2]

'start выступление женского хора черном end'

### Сконвертируем наши описания в словарь

In [26]:
def to_dict(data):
    out_dict = dict()

    start_index = 0
    end_index = len(data) - 1 - 5

    for idx in range(start_index, end_index+1, 5):
        image_name = data.iat[idx, 0][:-4]
        curr_list = list()

        curr_list.append(data.iat[idx, 2])
        curr_list.append(data.iat[idx+1, 2])
        curr_list.append(data.iat[idx+2, 2])
        curr_list.append(data.iat[idx+3, 2])
        curr_list.append(data.iat[idx+4, 2])

        out_dict[image_name] = curr_list

    return out_dict

In [27]:
# Сохраним словарь с валидационной выборкой в pkl
df_to_convert = pd.read_csv(path_val, sep='|')
df_to_convert.head()
new_dict = to_dict(df_to_convert)

path_val_dict = dataset_path + "/captions-ru-val.pkl"

with open(path_val_dict,'wb') as f:
    pickle.dump(new_dict, f)

with open (path_val_dict, 'rb') as f:
    test_dict = pickle.load(f)

# Сохраним словарь с обучающей выборкой в pkl
df_to_convert_train = pd.read_csv(path_train, sep='|')
df_to_convert_train.head()
train_dict = to_dict(df_to_convert_train)

path_train_dict = dataset_path + "/captions-ru-train.pkl"

with open(path_train_dict,'wb') as f:
    pickle.dump(train_dict, f)




# Предложения для обучения

In [39]:
def to_sentences(data):
    all_sentences = list()
    for idx in data.index:
       all_sentences.append(data.iat[idx, 2])
    return all_sentences

train_df = pd.read_csv(path_train, delimiter='|')
sentences = to_sentences(train_df)
print(len(sentences))

111237


In [46]:
path_sentences = dataset_path + "ru-12k-sentences-train.pkl"

with open(path_sentences, 'wb') as f:
 pickle.dump(sentences, f)

with open (path_sentences, 'rb') as f:
 list_sentences = pickle.load(f)

print(list_sentences[0])

start два молодых парня лохматыми волосами смотрят на свои руки болтаясь во дворе end


### Скопируем изображения

In [63]:
df = pd.read_csv(path_ru_captions, sep='|')
images_unique = df["image_name"].unique()
print(images_unique.shape)
print(df.shape)

(31782,)
(158910, 3)


In [64]:
start_dir = dataset_path + "/flickr30k_images"
end_dir = dataset_path + "/copy_flickr30k_images"

In [66]:
for image_name in images_unique:
    curr_image = start_dir + '/' + image_name
    copied_image = end_dir + '/' + image_name
    shutil.copy(curr_image, copied_image )

counter = len(os.listdir(path=end_dir))
print("скопировано изображений ... " + str(counter))

скопировано изображений ... 31782


## Скопируем изображения для обучаюзщей выборки

In [37]:
print(len(train_dict))
start_dir = dataset_path + "/flickr30k_images"
end_dir = dataset_path + "/copy_train_flickr30k_images"
for k in train_dict.keys():
    curr_train_image = start_dir + '/' + k + ".jpg"
    copied_train_image = end_dir + '/' + k + ".jpg"
    shutil.copy(curr_train_image, copied_train_image )

22247


## Токенизация словарного запаса из обучающей выборки

In [49]:
def to_lines(data):
    all_vals = list()
    print(data.keys())
    for key in data.keys():
        [all_vals.append(d) for d in data[key]]

    return all_vals

def create_tokenizer(data):
    lines = to_lines(data)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)

    return tokenizer

path_tokenizer = dataset_path + "/ru-tokenizer-train.pkl"

with open (path_train_dict, 'rb') as f:
        out_train_dict = pickle.load(f)
tokenizer = create_tokenizer(out_train_dict)

with open(path_tokenizer, 'wb') as f:
 pickle.dump(tokenizer, f)

dict_keys(['1000092795', '10002456', '1000268201', '1000344755', '1000366164', '1000523639', '1000919630', '10010052', '1001465944', '1001545525', '1001573224', '1001633352', '1001773457', '1001896054', '100197432', '100207720', '1002674143', '1003163366', '1003420127', '1003428081', '100444898', '1005216151', '100577935', '1006452823', '100652400', '1007129816', '100716317', '1007205537', '1007320043', '100759042', '10082347', '10082348', '100845130', '10090841', '1009434119', '1009692167', '101001624', '1010031975', '1010087179', '1010087623', '10101477', '1010470346', '1010673430', '101093029', '101093045', '1011572216', '1012150929', '1012212859', '1012328893', '101262930', '1013536888', '101362133', '101362650', '1014609273', '101471792', '1014785440', '1015118661', '1015584366', '101559400', '1015712668', '10160966', '101654506', '1016626169', '101669240', '1016887272', '1017675163', '1018057225', '1018148011', '101859883', '10188041', '1019077836', '101958970', '1019604187', '10